ResNet和DenseNet在跨层连接上的主要区别：使用相加和使用连结（在通道维上连结）

这样模块A的输出可以直接传入模块B后面的层，即模块A直接跟模块B后面的所有层连接在了一起。

DenseNet：Dense Block + Transition Layer，前者定义了输入和输出是如何连结的，后者则用来控制通道数，使之不过大

# Dense Block
和ResNet一样，采用批量归一化、激活和卷积的操作。

In [1]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
def conv_block(in_channels, out_channels):
    blk = nn.Sequential(
        nn.BatchNorm2d(in_channels),
        nn.ReLU(),
        nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
    )
    return blk

In [3]:
class DenseBlock(nn.Module):
    def __init__(self, num_convs,in_channels, out_channels):
        super(DenseBlock, self).__init__()
        net = []
        # 稠密结构的卷积几次
        for i in range(num_convs):
            in_c = in_channels + i * out_channels
            net.append(conv_block(in_c, out_channels))
        self.net = nn.ModuleList(net)
        # 计算输出通道数
        self.out_channels = in_channels + num_convs * out_channels
    
    def forward(self, X):
        for blk in self.net:
            Y = blk(X)
            X = torch.cat((X, Y), dim=1)
        return X

In [4]:
# 我们定义一个有2个输出通道数为10的卷积块，
# 最终会得到通道数为3+2*10=23的输出
# 卷积块的通道数控制了输出通道数相对于输入通道数的增长，out_channels被称为增长率
blk = DenseBlock(2, 3, 10)
X = torch.rand(4, 3, 8, 8)
Y = blk(X)
Y.shape

torch.Size([4, 23, 8, 8])

# 过渡层
因为通道数增加了，模型复杂度也就增加了，我们需要控制模型复杂度，通过1 * 1卷积层来减小通道数。并通过stride=2的平均池化层减半宽和高。

In [8]:
def transition_block(in_channels, out_channels):
    blk = nn.Sequential(
        nn.BatchNorm2d(in_channels),
        nn.ReLU(),
        nn.Conv2d(in_channels, out_channels, kernel_size=1),
        nn.AvgPool2d(kernel_size=2, stride=2)
    )
    return blk

In [9]:
blk = transition_block(23, 10)
blk(Y).shape

torch.Size([4, 10, 4, 4])

# DenseNet Model
首先和ResNet一样的单层卷积和MaxPool

In [5]:
net = nn.Sequential(
    nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
)

接下来是使用4个DenseBlock，num_convs=4, out_channels=32

In [11]:
num_channels, growth_rate = 64, 32
num_convs_in_dense_blocks = [4, 4, 4, 4]

for i, num_convs in enumerate(num_convs_in_dense_blocks):
    DB = DenseBlock(num_convs, num_channels, growth_rate)
    net.add_module('DenseBlock_%d' % i, DB)
    # 上一个DenseBlock的输出通道数
    num_channels = DB.out_channels
    # 在DenseBlock之间加入通道数 减半 的过渡层
    if i != len(num_convs_in_dense_blocks) - 1:
        net.add_module('transition_block_%d' % i, transition_block(num_channels, num_channels // 2))
        num_channels = num_channels // 2

In [12]:
# 最后连接上全局池化层+全连接层
net.add_module('BN', nn.BatchNorm2d(num_channels))
net.add_module('relu', nn.ReLU())
# GlobalAvgPool2d 输出（Batch，num_channels, 1, 1）
net.add_module('global_avg_pool', d2l.GlobalAvgPool2d())
net.add_module('fc', nn.Sequential(d2l.FlattenLayer(), nn.Linear(num_channels, 10)))

In [13]:
X = torch.rand((1, 1, 96, 96))
for name, layer in net.named_children():
    X = layer(X)
    print(name, ' output shape:\t', X.shape)

0  output shape:	 torch.Size([1, 64, 48, 48])
1  output shape:	 torch.Size([1, 64, 48, 48])
2  output shape:	 torch.Size([1, 64, 48, 48])
3  output shape:	 torch.Size([1, 64, 24, 24])
DenseBlock_0  output shape:	 torch.Size([1, 192, 24, 24])
transition_block_0  output shape:	 torch.Size([1, 96, 12, 12])
DenseBlock_1  output shape:	 torch.Size([1, 224, 12, 12])
transition_block_1  output shape:	 torch.Size([1, 112, 6, 6])
DenseBlock_2  output shape:	 torch.Size([1, 240, 6, 6])
transition_block_2  output shape:	 torch.Size([1, 120, 3, 3])
DenseBlock_3  output shape:	 torch.Size([1, 248, 3, 3])
BN  output shape:	 torch.Size([1, 248, 3, 3])
relu  output shape:	 torch.Size([1, 248, 3, 3])
global_avg_pool  output shape:	 torch.Size([1, 248, 1, 1])
fc  output shape:	 torch.Size([1, 10])
